# Process Tail Simulation Results

In [9]:
import os, pickle

results_dir = r"D:\Ergodicity Simulations\2025-10-05 Run - Varying Loss Tail and Limits"

sample_file = "Ded (100K) - LR (0.3) - Pol_Lim (50M) - X_Th_%le (0.0001) - X_Shape (1.0) - X_Scale (1.0) - 100K Sims - 25 Yrs.pkl"

file_path = os.path.join(results_dir, sample_file)
with open(file_path, "rb") as f:
    loaded_data = pickle.load(f)

In [10]:
loaded_data

SimulationResults(final_assets=array([9.9838340e+06, 9.7759760e+06, 1.0036777e+07, ..., 9.9885040e+06,
       1.0295064e+07, 9.8392280e+06], shape=(100000,), dtype=float32), annual_losses=array([[2.14245176e+04, 6.60459688e+05, 2.30505094e+05, ...,
        1.17622203e+05, 2.21924062e+04, 1.96936406e+05],
       [2.02290797e+05, 9.16902812e+04, 1.29339625e+05, ...,
        0.00000000e+00, 2.51244141e+05, 2.40998125e+05],
       [4.73054805e+04, 1.29710156e+05, 3.53662266e+04, ...,
        1.65521656e+05, 1.81192012e+06, 2.16245125e+05],
       ...,
       [1.48980578e+05, 1.07651609e+05, 1.08089875e+05, ...,
        1.36311016e+05, 8.50685000e+04, 3.24444250e+05],
       [4.89740594e+05, 3.89913516e+04, 2.06956391e+05, ...,
        2.04392078e+05, 8.49553672e+04, 1.69961656e+05],
       [1.43140219e+05, 9.66421719e+04, 2.53419016e+05, ...,
        8.87786953e+04, 4.67316094e+04, 7.85095703e+04]],
      shape=(100000, 25), dtype=float32), insurance_recoveries=array([[0.0000000e+00, 4.660

In [7]:
loaded_data.ruin_probability

{'5': 0.00016, '10': 0.00034, '15': 0.00046, '20': 0.00056, '25': 0.00076}

In [8]:
loaded_data.growth_rates.mean()

np.float64(-0.00010129731595399839)

In [13]:
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults
import numpy as np

def _parse_number(text):
    text = text.strip().replace(",", "")
    m = re.fullmatch(r'([+-]?\d+(?:\.\d+)?)([KMB])?$', text, re.I)
    if not m:
        # fallback: plain int/float or leave as-is
        try:
            return int(text)
        except ValueError:
            try:
                return float(text)
            except ValueError:
                return text
    num = float(m.group(1))
    mult = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}.get((m.group(2) or "").upper(), 1)
    val = num * mult
    return int(val) if val.is_integer() else val


def parse_config_key(key: str) -> dict:
    parts = re.split(r"\s*-\s*", key.strip())
    out = {}
    for part in parts:
        if not part:
            continue

        # e.g. "Cap (100M)"
        m = re.match(r"^([A-Za-z_%]+)\s*\(\s*([^)]+)\s*\)$", part)
        if m:
            out[m.group(1)] = _parse_number(m.group(2))
            continue

        # e.g. "0K Sims" or "50 Yrs"
        m = re.match(r"^([+-]?\d+(?:\.\d+)?)\s*([KMB])?\s*([A-Za-z_]+)$", part)
        if m:
            value = _parse_number((m.group(1) or "") + (m.group(2) or ""))
            out[m.group(3)] = value
            continue

        # flags like "NOINS"
        if part.upper() == "NOINS":
            out["NOINS"] = True

    return out

results_dir = Path(results_dir)

pkl_paths = sorted(results_dir.glob(sample_file))

qs = (0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99) # Growth Rate Quantiles

all_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    for path in pkl_paths:
        try:
            with open(path, "rb") as f:
                one_config = pickle.load(f)
                growth_rate = one_config.growth_rates.mean()
                growth_rate_ci = {str(q): val for q, val in zip(qs, np.quantile(one_config.growth_rates, qs))}
                ror = one_config.ruin_probability
                all_configurations[path.stem] = {
                    "growth_rate": growth_rate,
                    "growth_rate_ci": growth_rate_ci,
                    "risk_of_ruin": ror
                }
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(all_configurations)} pickle files into all_configurations.")

parsed_params_by_key = {k: parse_config_key(k) | all_configurations[k] for k in all_configurations.keys()}
parsed_params_by_key

Loaded 1 pickle files into all_configurations.


{'Ded (100K) - LR (0.3) - Pol_Lim (50M) - X_Th_%le (0.0001) - X_Shape (1.0) - X_Scale (1.0) - 100K Sims - 25 Yrs': {'Ded': 100000,
  'LR': 0.3,
  'Pol_Lim': 50000000,
  'X_Th_%le': 0.0001,
  'X_Shape': 1,
  'X_Scale': 1,
  'Sims': 100000,
  'Yrs': 25,
  'growth_rate': np.float64(-0.00010129731595399839),
  'growth_rate_ci': {'0.5': np.float64(-1.495283140684478e-05),
   '0.6': np.float64(0.00020352471619844437),
   '0.7': np.float64(0.0004352374584414064),
   '0.8': np.float64(0.0006972453324124219),
   '0.9': np.float64(0.001034455699846149),
   '0.95': np.float64(0.0012995375611353653),
   '0.99': np.float64(0.0017356308898888518)},
  'risk_of_ruin': {'5': 0.00016,
   '10': 0.00034,
   '15': 0.00046,
   '20': 0.00056,
   '25': 0.00076}}}